## Natural Language to SQL
1. Building a basic NL2SQL Model
2. Adding few-shot examples
3. Dynamic few-shot example selection
4. Dynamic Relevant table selection
5. Customizing prompts
6. Adding memory to the chatbots so that it can answer follow up questions related to DB

### Building a Basic NL2SQL Model

In [1]:
from langchain_community.utilities.sql_database import SQLDatabase
from langchain.chains import create_sql_query_chain
from langchain_community.tools.sql_database.tool import QuerySQLDataBaseTool
from langchain_openai import ChatOpenAI
from dotenv import load_dotenv

In [2]:
# load the environment variables from .env file
load_dotenv()

True

In [3]:
# Connect the database 
db = SQLDatabase.from_uri("sqlite:///db/company.sqlite3")

/Users/jithinjose/Documents/TechincalProjects/PythonProjects/Project Langchain/NL2SQL/.conda/lib/python3.9/site-packages/langchain_community/utilities/sql_database.py:123: SAWarning: Could not instantiate type <class 'sqlalchemy.sql.sqltypes.SMALLINT'> with reflected arguments ['6']; using no arguments.
  self._metadata.reflect(


In [4]:
# get the basic info from the database and tables
print(db.dialect)
print(db.get_usable_table_names())
print(db.table_info)

sqlite
['customers', 'employees', 'offices', 'orderdetails', 'orders', 'payments', 'productlines', 'products']

CREATE TABLE customers (
	"customerNumber" INTEGER, 
	"customerName" VARCHAR(50) NOT NULL, 
	"contactLastName" VARCHAR(50) NOT NULL, 
	"contactFirstName" VARCHAR(50) NOT NULL, 
	phone VARCHAR(50) NOT NULL, 
	"addressLine1" VARCHAR(50) NOT NULL, 
	"addressLine2" VARCHAR(50) DEFAULT NULL, 
	city VARCHAR(50) NOT NULL, 
	state VARCHAR(50) DEFAULT NULL, 
	"postalCode" VARCHAR(15) DEFAULT NULL, 
	country VARCHAR(50) NOT NULL, 
	"salesRepEmployeeNumber" INTEGER DEFAULT NULL, 
	"creditLimit" DECIMAL(10, 2) DEFAULT NULL, 
	PRIMARY KEY ("customerNumber"), 
	FOREIGN KEY("salesRepEmployeeNumber") REFERENCES employees ("employeeNumber")
)

/*
3 rows from customers table:
customerNumber	customerName	contactLastName	contactFirstName	phone	addressLine1	addressLine2	city	state	postalCode	country	salesRepEmployeeNumber	creditLimit
103	Atelier graphique	Schmitt	Carine 	40.32.2555	54, rue Royale

In [5]:
# define llm
llm = ChatOpenAI(model="gpt-3.5-turbo", temperature=0, verbose=True)

In [6]:
# Generate query chain
generate_query = create_sql_query_chain(llm, db)

# invoke the chain to get the query
query = generate_query.invoke({"question": "What is the price of 1968 Ford Mustang like car?"})
# print("--- Generated Query ----")
print(query)
# print("--------------")

--- Generated Query ----
SELECT "buyPrice", "MSRP"
FROM products
WHERE "productName" LIKE '%1968 Ford Mustang%'
--------------


In [7]:
execute_query = QuerySQLDataBaseTool(db=db)
result = execute_query.invoke(query)
print(result)

[(95.34, 194.57)]


In [8]:
# Define chain using LCEL
chain = generate_query | execute_query
chain.invoke({"question": "How many orders are there?"})

'[(326,)]'

In [9]:
# View the underhood prompt from the chain
chain.get_prompts() [0].pretty_print()

You are a SQLite expert. Given an input question, first create a syntactically correct SQLite query to run, then look at the results of the query and return the answer to the input question.
Unless the user specifies in the question a specific number of examples to obtain, query for at most 5 results using the LIMIT clause as per SQLite. You can order the results to return the most informative data in the database.
Never query for all columns from a table. You must query only the columns that are needed to answer the question. Wrap each column name in double quotes (") to denote them as delimited identifiers.
Pay attention to use only the column names you can see in the tables below. Be careful to not query for columns that do not exist. Also, pay attention to which column is in which table.
Pay attention to use date('now') function to get the current date, if the question involves "today".

Use the following format:

Question: Question here
SQLQuery: SQL Query to run
SQLResult: Result

In [10]:
# Rephrase the answer
from operator import itemgetter
from langchain_core.output_parsers import StrOutputParser
from langchain_core.prompts import PromptTemplate
from langchain_core.runnables import RunnablePassthrough

answer_prompt = PromptTemplate.from_template(
    """Given the following user question, corresponding SQL query, and SQL result, answer the user question
    
    Question: {question}
    SQL Query: {query}
    SQL Result: {result}
    Answer: """
)

rephrase_answer = answer_prompt | llm | StrOutputParser()
chain = (
    RunnablePassthrough.assign(query=generate_query).assign(result=itemgetter("query") | execute_query) | rephrase_answer
)

chain.invoke({"question": "How many orders are there?"})

'There are a total of 326 orders.'

### Adding few-shot examples

In [11]:
examples = [
    {
        "input": "List all customers in France with a credit limit over 20,000.",
        "query": "SELECT * FROM customers WHERE country = 'France' AND creditLimit > 20000;"
    },
    {
        "input": "Get the highest payment amount made by any customer.",
        "query": "SELECT MAX(amount) FROM payments;"
    },
    {
        "input": "Show product details for products in the 'Motorcycles' product line.",
        "query": "SELECT * FROM products WHERE productLine = 'Motorcycles';"
    },
    {
        "input": "Retrieve the names of employees who report to employee number 1002.",
        "query": "SELECT firstName, lastName FROM employees WHERE reportsTo = 1002;"
    },
    {
        "input": "List all products with a stock quantity less than 7000.",
        "query": "SELECT productName, quantityInStock FROM products WHERE quantityInStock < 7000;"
    },
    {
     'input':"what is price of `1968 Ford Mustang`",
     "query": "SELECT `buyPrice`, `MSRP` FROM products  WHERE `productName` = '1968 Ford Mustang' LIMIT 1;"
    }
]

In [15]:
from langchain_core.prompts import ChatPromptTemplate, MessagesPlaceholder, FewShotChatMessagePromptTemplate,PromptTemplate

example_prompt = ChatPromptTemplate.from_messages(
    [
        ("human","{input}\nSQLQuery:"),
        ("ai","{query}")
    ]
)
few_shot_prompt = FewShotChatMessagePromptTemplate(
    example_prompt=example_prompt,
    examples=examples,
    # input_variables=["input", "top_k"],
    input_variables=["input"],
)

# print(few_shot_prompt.format(input="How many products are there?", top_k=3, table_info=""))
print(few_shot_prompt.format(input="How many products are there?"))

Human: List all customers in France with a credit limit over 20,000.
SQLQuery:
AI: SELECT * FROM customers WHERE country = 'France' AND creditLimit > 20000;
Human: Get the highest payment amount made by any customer.
SQLQuery:
AI: SELECT MAX(amount) FROM payments;
Human: Show product details for products in the 'Motorcycles' product line.
SQLQuery:
AI: SELECT * FROM products WHERE productLine = 'Motorcycles';
Human: Retrieve the names of employees who report to employee number 1002.
SQLQuery:
AI: SELECT firstName, lastName FROM employees WHERE reportsTo = 1002;
Human: List all products with a stock quantity less than 7000.
SQLQuery:
AI: SELECT productName, quantityInStock FROM products WHERE quantityInStock < 7000;
Human: what is price of `1968 Ford Mustang`
SQLQuery:
AI: SELECT `buyPrice`, `MSRP` FROM products  WHERE `productName` = '1968 Ford Mustang' LIMIT 1;


### Dynamic Few-shot example selection

In [18]:
from langchain_community.vectorstores import Chroma
from langchain_core.example_selectors import SemanticSimilarityExampleSelector
from langchain_openai import OpenAIEmbeddings

vectorstore = Chroma()
vectorstore.delete_collection()
example_selector = SemanticSimilarityExampleSelector.from_examples(
    examples=examples,
    embeddings=OpenAIEmbeddings(),
    vectorstore_cls=vectorstore,
    k=2,
    input_keys=["input"]
)

example_selector.select_examples({"input": "how many employees we have?"})
# example_selector.select_examples({"input": "how many employees"})

[{'input': 'Retrieve the names of employees who report to employee number 1002.',
  'query': 'SELECT firstName, lastName FROM employees WHERE reportsTo = 1002;'},
 {'input': 'List all customers in France with a credit limit over 20,000.',
  'query': "SELECT * FROM customers WHERE country = 'France' AND creditLimit > 20000;"}]

In [19]:
# modify the few_shot_prompt with example selector
few_shot_prompt = FewShotChatMessagePromptTemplate(
    example_prompt=example_prompt,
    example_selector=example_selector,
    input_variables=["input","top_k"]
)

print(few_shot_prompt.format(input="How many products are there?"))

Human: List all products with a stock quantity less than 7000.
SQLQuery:
AI: SELECT productName, quantityInStock FROM products WHERE quantityInStock < 7000;
Human: Show product details for products in the 'Motorcycles' product line.
SQLQuery:
AI: SELECT * FROM products WHERE productLine = 'Motorcycles';


### Customizing the Prompts

In [20]:
final_prompt = ChatPromptTemplate.from_messages([
    ("system", "You are a MySQL expert. Given an input question, create a syntactically correct MySQL query to run. Unless otherwise specificed.\n\nHere is the relevant table info: {table_info}\n\nBelow are a number of examples of questions and their corresponding SQL queries."),
    few_shot_prompt,
    ("human","{input}"),

])

print(final_prompt.format(input="How many products are there?", table_info="some table info"))

System: You are a MySQL expert. Given an input question, create a syntactically correct MySQL query to run. Unless otherwise specificed.

Here is the relevant table info: some table info

Below are a number of examples of questions and their corresponding SQL queries.
Human: List all products with a stock quantity less than 7000.
SQLQuery:
AI: SELECT productName, quantityInStock FROM products WHERE quantityInStock < 7000;
Human: Show product details for products in the 'Motorcycles' product line.
SQLQuery:
AI: SELECT * FROM products WHERE productLine = 'Motorcycles';
Human: How many products are there?


In [21]:
generate_query = create_sql_query_chain(llm=llm, db=db, prompt=final_prompt)
chain = (
    RunnablePassthrough.assign(query=generate_query).assign(result=itemgetter("query") | execute_query ) | rephrase_answer
    )

chain.invoke({"question": "How many customers with credit limit more than 50000"})

'There are 85 customers with a credit limit greater than 50000.'

### Dynamic Table Selection